![https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F3244747%2Fdf2bd9836c198215ed033d0678a02ef2%2Fheader.png?generation=1604778403014926&alt=media](https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F3244747%2Fdf2bd9836c198215ed033d0678a02ef2%2Fheader.png?generation=1604778403014926&alt=media)

In [1]:
!pip install --upgrade plotly-geo
!pip install --upgrade geopandas
!pip install --upgrade pyshp
!pip install --upgrade shapely

     |████████████████████████████████| 23.7 MB 209 kB/s 
Requirement already up-to-date: geopandas in /opt/conda/lib/python3.7/site-packages (0.8.1)
Requirement already up-to-date: pyshp in /opt/conda/lib/python3.7/site-packages (2.1.2)
Requirement already up-to-date: shapely in /opt/conda/lib/python3.7/site-packages (1.7.1)


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.figure_factory as ff
import plotly.express as px
import time
from datetime import datetime

## US Voting Results

In [3]:
datafile = pd.read_csv('../input/us-election-2020/president_state.csv')
state_codes = pd.read_csv('../input/coordinates/world_country_and_usa_states_latitude_and_longitude_values.csv')
datafile = datafile.merge(state_codes, left_on='state', right_on='usa_state')

In [4]:
fig = px.choropleth(datafile, locations='usa_state_code', color="votes",
                           range_color=(0, 10000000),
                           locationmode = 'USA-states',  
                           scope="usa",
                           title='USA Presidential Votes Counts' 
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## 2020 USA Election: Vote Percentages by State

In [5]:
#referennce : https://www.kaggle.com/paultimothymooney/2020-usa-election-vote-percentages-by-state

df_president_county = pd.read_csv('/kaggle/input/us-election-2020/president_county_candidate.csv')
democrat = df_president_county['party'] == 'DEM'
republican = df_president_county['party'] == 'REP'
df_president_county = df_president_county[democrat|republican]
df_president_county = df_president_county.groupby(['state','party']).sum()
df_president_county = df_president_county.unstack()
df_president_county.columns = ['DEM','REP']
df_president_county = df_president_county.reset_index()

state_codes = pd.read_csv('/kaggle/input/coordinates/world_country_and_usa_states_latitude_and_longitude_values.csv')
state_codes = state_codes[['usa_state','usa_state_code']]
df_president_county = df_president_county.merge(state_codes, left_on='state', right_on='usa_state')
df_president_county['percent_democrat'] = df_president_county['DEM']*100/(df_president_county['REP']+df_president_county['DEM'])

In [6]:
fig = px.choropleth(df_president_county, 
                    locations="usa_state_code", 
                    color = "percent_democrat",
                    locationmode = 'USA-states', 
                    hover_name="state",
                    range_color=[25,75],
                    color_continuous_scale = 'RdBu',#blues
                    scope="usa",
                    title='2020 USA Election: Percent of Population Voting for the Democratic Party')
fig.show()

## Governor's County Results

In [7]:
flips_main = pd.read_csv('../input/flips-for-maps/Flip_v2.csv')
flips = flips_main[['State','County Full Name','FIPS Full County']]
flips.columns = ['state','county','Flips']

In [8]:
flips.head(2)

,state,county,Flips
0,Alabama,Autauga County,1001
1,Alabama,Baldwin County,1003


In [9]:
flips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   state   3142 non-null   object
 1   county  3142 non-null   object
 2   Flips   3142 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 73.8+ KB


In [10]:
gov_county = pd.read_csv('../input/us-election-2020/governors_county.csv')
gov_county.head(2)

,state,county,current_votes,total_votes,percent
0,Delaware,Kent County,85399,88000,95
1,Delaware,New Castle County,278691,293878,95


## Delaware

In [11]:
Delaware_County = gov_county[gov_county['state'] == 'Delaware']
Delaware_Flips = flips[flips['state']=='Delaware']

In [12]:
Delaware_County = Delaware_County.merge(Delaware_Flips, on='county', how='left')
Delaware_County = Delaware_County.drop(['state_y'],axis=1)
Delaware_County.head(2)

,state_x,county,current_votes,total_votes,percent,Flips
0,Delaware,Kent County,85399,88000,95,10001
1,Delaware,New Castle County,278691,293878,95,10003


In [13]:
county = Delaware_County['county'].tolist()
values = Delaware_County['current_votes'].tolist()
fips = Delaware_County['Flips'].tolist()

endpts = list(np.mgrid[min(values):max(values):4j])

fig = ff.create_choropleth(
    fips=fips, values=values, scope=['Delaware'], show_state_data=True,
    binning_endpoints=endpts, round_legend_values=True,
    plot_bgcolor='rgb(229,229,229)',
    paper_bgcolor='rgb(229,229,229)',
    legend_title='Total Votes by County',
    show_hover=True,
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
    exponent_format=True,
    title='Delaware'
)
fig.update_layout(height=400, width=750, margin={"r":0,"t":0,"l":0,"b":0})
fig.layout.template = None
fig.show()

## Indiana

In [14]:
Indiana_County = gov_county[gov_county['state'] == 'Indiana']
Indiana_Flips = flips[flips['state']=='Indiana']

In [15]:
Indiana_Flips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92 entries, 694 to 785
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   state   92 non-null     object
 1   county  92 non-null     object
 2   Flips   92 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 2.9+ KB


In [16]:
Indiana_County = Indiana_County.merge(Indiana_Flips, on='county', how='left')
Indiana_County = Indiana_County.drop(['state_y'],axis=1)
Indiana_County.head(2)

,state_x,county,current_votes,total_votes,percent,Flips
0,Indiana,Adams County,14153,14238,99,18001
1,Indiana,Allen County,140363,164793,85,18003


In [17]:
county = Indiana_County['county'].tolist()
values = Indiana_County['current_votes'].tolist()
fips = Indiana_County['Flips'].tolist()

endpts = list(np.mgrid[min(values):max(values):4j])

fig = ff.create_choropleth(
    fips=fips, values=values, scope=['Indiana'], show_state_data=True,
    binning_endpoints=endpts, round_legend_values=True,
    plot_bgcolor='rgb(229,229,229)',
    paper_bgcolor='rgb(229,229,229)',
    legend_title='Total Votes by County',
    show_hover=True,
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
    exponent_format=True,
    title='Indiana'
)
fig.update_layout(height=400, width=750, margin={"r":0,"t":0,"l":0,"b":0})
fig.layout.template = None
fig.show()

## Missouri

In [18]:
Missouri_County = gov_county[gov_county['state'] == 'Missouri']
Missouri_Flips = flips[flips['state']=='Missouri']

In [19]:
Missouri_County = Missouri_County.merge(Missouri_Flips, on='county', how='left')
Missouri_County = Missouri_County.drop(['state_y'],axis=1)
Missouri_County.head(2)

,state_x,county,current_votes,total_votes,percent,Flips
0,Missouri,Adair County,10292,10337,99,29001
1,Missouri,Andrew County,9733,9774,99,29003


In [20]:
county = Missouri_County['county'].tolist()
values = Missouri_County['current_votes'].tolist()
fips = Missouri_County['Flips'].tolist()

endpts = list(np.mgrid[min(values):max(values):4j])

fig = ff.create_choropleth(
    fips=fips, values=values, scope=['Missouri'], show_state_data=True,
    binning_endpoints=endpts, round_legend_values=True,
    plot_bgcolor='rgb(229,229,229)',
    paper_bgcolor='rgb(229,229,229)',
    legend_title='Total Votes by County',
    show_hover=True,
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
    exponent_format=True,
    title='Missouri'
)
fig.update_layout(height=400, width=750, margin={"r":0,"t":0,"l":0,"b":0})
fig.layout.template = None
fig.show()

## Montana

In [21]:
Montana_County = gov_county[gov_county['state'] == 'Montana']
Montana_Flips = flips[flips['state']=='Montana']

In [22]:
Montana_County = Montana_County.merge(Montana_Flips, on='county', how='left')
Montana_County = Montana_County.drop(['state_y'],axis=1)
Montana_County.head(2)

,state_x,county,current_votes,total_votes,percent,Flips
0,Montana,Beaverhead County,5655,5664,99,30001
1,Montana,Big Horn County,4677,5191,90,30003


In [23]:
county = Montana_County['county'].tolist()
values = Montana_County['current_votes'].tolist()
fips = Montana_County['Flips'].tolist()

endpts = list(np.mgrid[min(values):max(values):4j])

fig = ff.create_choropleth(
    fips=fips, values=values, scope=['Montana','North Dakota'], show_state_data=True,
    binning_endpoints=endpts, round_legend_values=True,
    plot_bgcolor='rgb(229,229,229)',
    paper_bgcolor='rgb(229,229,229)',
    legend_title='Total Votes by County',
    show_hover=True,
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
    exponent_format=False,
    title='Montana'
)
fig.update_layout(height=400, width=750, margin={"r":0,"t":0,"l":0,"b":0})
fig.layout.template = None
fig.show()

## Come back for more updates!!